In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.ndimage.measurements import center_of_mass
from tqdm import tqdm_notebook as tqdm
import h5py

In [ ]:
"""
Extract images from npy files
"""
train_x = np.load('./datasets/original/train_images.npy', encoding='latin1')
train_x = np.array([arr[1] for arr in train_x], np.uint8)
train_x = np.array([train_x[idx].reshape(100, 100) for idx in range(len(train_x))], np.uint8)
train_y = np.array(pd.read_csv('./datasets/original/train_labels.csv', usecols=['Category']))

test_x = np.load('./datasets/original/test_images.npy', encoding='latin1')
test_x = np.array([arr[1] for arr in test_x], np.uint8)
test_x = np.array([test_x[idx].reshape(100, 100) for idx in range(len(test_x))], np.uint8)

In [ ]:
def find_contours(img_gray):
    """
    Finding contours for the
    given greyscale image

    @param
        :type np.array img_gray - greyscale version of the image

    @return
        :rtype np.array - the image
        :rtype np.array - contours found for the image

    """
    ret, thresh = cv2.threshold(img_gray, 127, 255, 0)
    thresh = thresh.astype('uint8')
    image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return image, contours

In [ ]:
def clean_noise_by_contours(image, contours):
    """
    Cleaning out the noise from the
    image based on the contours found
    using find_contours(img_gray)

    @param
        :type np.array image - image to clean
        :type np.arary contours - contours found for the image

    @return
        :rtpe np.array - cleaned version of the image
        
    """
    num_contour = len(contours)
    
    max_area = 0
    max_area_idx = 0
    for i in range(0, num_contour):
        contour = contours[i]
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_area_idx = i
    
    for i in range(0, num_contour):
        if i != max_area_idx:
            cv2.drawContours(image, contours, i, -1, 3)
            
    return image

In [ ]:
def cropImage(cleaned_img):
    """
    Crop the cleaned image to
    return just the drawing
    in a smaller square of side 30.

    @param
        :type np.array cleaned_img - cleaned image

    @return
        :rtype np.array - cropped version of the cleaned image
        
    """
    sq_side_length = 30
    sq_half_side = int(sq_side_length/2)
    center_h, center_w = center_of_mass(cleaned_img)
    center_h = int(center_h)
    center_w = int(center_w)
    top_h = center_h - sq_half_side
    bottom_h = center_h + sq_half_side
    left_w = center_w - sq_half_side
    right_w = center_w + sq_half_side
    
    new_val_bound = check_within_bounds(top_h)
    top_h = new_val_bound[0]
    bottom_h += new_val_bound[1]
    
    new_val_bound = check_within_bounds(bottom_h)
    bottom_h = new_val_bound[0]
    top_h -= new_val_bound[1]
    
    new_val_bound = check_within_bounds(left_w)
    left_w = new_val_bound[0]
    right_w += new_val_bound[1]
    
    new_val_bound = check_within_bounds(right_w)
    right_w = new_val_bound[0]
    left_w -= new_val_bound[1]
    
    cropped_img = cleaned_img[top_h: bottom_h, left_w: right_w]
    return cropped_img

In [ ]:
def check_within_bounds(val_to_check):
    """
    Check if the x or y coordinate is 
    within the 100x100 bounds of the image
    
    @param
        :type int val_to_check - coordinate value
    
    @return
        :rtype int - new value for the coordinate
        :rtype int - magnitude of overflow beyond the boundary
        
    """
    if val_to_check < 0:
        return 0, 0 - val_to_check
    elif val_to_check > 100:
        return 100, val_to_check - 100
    else:
        return val_to_check, 0

In [ ]:
def rotate(img, angle):
    """
    Rotates the given image 
    by 'angle' degrees

    @param
        :type np.array img - image to rotate
        :type int angle - angle in degrees to rotate by

    @return
        :rtype np.array - original image rotated by 'angle' degrees
        
    """
    rows, cols = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2), angle, 1)
    dst = cv2.warpAffine(img, M, (cols,rows))
    return dst

In [ ]:
#### Don't have to run anymore. Train preprocessed data is saved ####
"""
Generating train preprocessed images in vector form
"""
train_processed_images = []
train_count = 0
train_vector_dim = 900
for train_pic in tqdm(train_x):
    train_image, train_contours = find_contours(train_pic)
    train_cleaned_img = clean_noise_by_contours(train_image, train_contours)
    train_cropped_img = cropImage(train_cleaned_img)
    train_processed_images.append(train_cropped_img.reshape(train_vector_dim,))
    train_count += 1

train_processed_images = np.array(train_processed_images)

In [ ]:
tpd = np.concatenate((train_processed_images, train_y), axis=1)

In [ ]:
np.random.shuffle(tpd)

In [ ]:
train_tenK_images = tpd[:, :900].astype('uint8')
train_tenK_labels = tpd[:, 900:901]

In [ ]:
"""
Saving the preprocessed images
and the updated labels to new files
"""
with h5py.File("./datasets/train_vectors_tenK.hdf5", "w") as h:
    dset = h.create_dataset("train_preprocessed", data=train_tenK_images)

df = pd.DataFrame(train_tenK_labels, columns=['Category'])
df.to_csv('./datasets/train_tenK_vectors_labels.csv', index_label='Id')

In [ ]:
"""
Read in the saved train data
"""
g = h5py.File('./datasets/train_vectors_tenK.hdf5', 'r')
train_preproc_x = np.array(g['train_preprocessed'])
train_preproc_y = np.array(pd.read_csv('./datasets/train_tenK_vectors_labels.csv', usecols=['Category']))

In [ ]:
#### Don't have to run anymore. Test preprocessed data is saved ####
"""
Generating test preprocessed images in vector form
"""
test_processed_images = []
test_count = 0
test_vector_dim = 900
for test_pic in tqdm(test_x):
    test_image, test_contours = find_contours(test_pic)
    test_cleaned_img = clean_noise_by_contours(test_image, test_contours)
    test_cropped_img = cropImage(test_cleaned_img)
    test_processed_images.append(test_cropped_img.reshape(test_vector_dim,))
    test_count += 1

test_processed_images = np.array(test_processed_images)

In [ ]:
with h5py.File("./datasets/test_vectors_tenK.hdf5", "w") as p:
    dset = p.create_dataset("test_preprocessed", data=test_processed_images)

In [ ]:
"""
Read in saved test data
"""
p = h5py.File('./datasets/test_vectors_tenK.hdf5', 'r')
test_preproc_x = np.array(p['test_preprocessed'])

In [ ]:
test_preproc_x.shape

In [ ]:
#### Don't have to run anymore. Train preprocessed data is saved ####
"""
Generating the training images in 30x30 format
"""
train_processed_images = []
train_count = 0
for train_pic in tqdm(train_x):
    train_image, train_contours = find_contours(train_pic)
    train_cleaned_img = clean_noise_by_contours(train_image, train_contours)
    train_cropped_img = cropImage(train_cleaned_img)
    train_processed_images.append(train_cropped_img)
    train_count += 1

train_processed_images = np.array(train_processed_images)

In [ ]:
train_tenK_pixels_images = train_processed_images
train_tenK_pixels_labels = train_y

In [ ]:
"""
Saving the preprocessed images
and the updated labels to new files
"""
with h5py.File("./datasets/train_pixels_tenK.hdf5", "w") as h:
    dset = h.create_dataset("train_preprocessed", data=train_tenK_pixels_images)

df = pd.DataFrame(train_tenK_pixels_labels, columns=['Category'])
df.to_csv('./datasets/train_tenK_pixels_labels.csv', index_label='Id')

In [ ]:
#### Don't have to run anymore. Test preprocessed data is saved ####
"""
Generating test preprocessed images in 30x30 format
"""
test_processed_images = []
test_count = 0
for test_pic in tqdm(test_x):
    test_image, test_contours = find_contours(test_pic)
    test_cleaned_img = clean_noise_by_contours(test_image, test_contours)
    test_cropped_img = cropImage(test_cleaned_img)
    test_processed_images.append(test_cropped_img)
    test_count += 1

test_processed_images = np.array(test_processed_images)

In [ ]:
test_tenK_pixels_images = test_processed_images

In [ ]:
"""
Saving the preprocessed images
and the updated labels to new files
"""
with h5py.File("./datasets/test_pixels_tenK.hdf5", "w") as h:
    dset = h.create_dataset("test_preprocessed", data=test_tenK_pixels_images)

In [ ]:
#### Don't have to run anymore. Train preprocessed data is saved ####
"""
Generating the training images in 100x100 format
"""
train_processed_images = []
train_count = 0
for train_pic in tqdm(train_x):
    train_image, train_contours = find_contours(train_pic)
    train_cleaned_img = clean_noise_by_contours(train_image, train_contours)
    train_processed_images.append(train_cleaned_img)
    train_count += 1

train_processed_images = np.array(train_processed_images)

In [ ]:
"""
Saving images:
Train
10k images
100x100
Cleaned
"""
with h5py.File("./datasets/train_cleaned_tenK.hdf5", "w") as h:
    dset = h.create_dataset("train_preprocessed", data=train_processed_images)

In [ ]:
#### Don't have to run anymore. test preprocessed data is saved ####
"""
Generating the testing images in 100x100 format
"""
test_processed_images = []
test_count = 0
for test_pic in tqdm(test_x):
    test_image, test_contours = find_contours(test_pic)
    test_cleaned_img = clean_noise_by_contours(test_image, test_contours)
    test_processed_images.append(test_cleaned_img)
    test_count += 1

test_processed_images = np.array(test_processed_images)

In [ ]:
"""
Saving images:
Test
10k images
100x100
Cleaned
"""
with h5py.File("./datasets/test_cleaned_tenK.hdf5", "w") as h:
    dset = h.create_dataset("test_preprocessed", data=test_processed_images)